Things to be done:
1. Use the GPNN feature files.
2. Format the GPNN feature files to include the bounding boxes, i3d features, 2d cnn features, geometric_feature,
motion of the central frame only.
3. 

In [12]:
import numpy as np
import torchvision
import torch
from copy import deepcopy as copy
import os

In [13]:
def feature_formatter(full_feature):

    new_feat = {}
    
    new_feat['i3d_feature_map'] = full_feature['i3d_feature_map'][0, :, 2, :, :]
    new_feat['2d_cnn_feature_map'] = full_feature['2d_cnn_feature_map'][ 5, :, :, :]
    
    new_feat['bboxes'] = full_feature['bboxes'][:, 5, :]
    
    req_keys = ['metadata', 'num_obj', 'lr', 'mr', 'cr', 'object_pairs', 'num_relation']
    
    for r in req_keys:
      new_feat[r] = full_feature[r]
    
    del full_feature
    return new_feat

In [14]:
from glob import glob

full_features_folder = '/workspace/data/data_folder/o2o/ral_features/full_features'
all_feature_files = glob( os.path.join(full_features_folder, '*.pt') )

from tqdm import tqdm as tqdm
import torch

num_all_features = len(all_feature_files)
saving_dir = '/workspace/data/data_folder/o2o/ral_features/formatted_features/ican'

for f in tqdm(all_feature_files):
    
    file_name = f.split('/')[-1]
    saving_path = os.path.join(saving_dir, file_name)
    
    if os.path.exists(saving_path):
        continue
        
    feature_dict = torch.load(f)
    formatted_dict = feature_formatter(feature_dict)
    torch.save(formatted_dict, saving_path)
    

    del feature_dict

100%|██████████| 2056/2056 [04:50<00:00,  7.07it/s]  


In [9]:
from glob import glob
import os

full_features_folder = '/workspace/data/data_folder/o2o/ral_features/formatted_features/ican'
all_feature_files = glob( os.path.join(full_features_folder, '*.pt') )

from tqdm import tqdm as tqdm
import torch

issues = []

for f in tqdm(all_feature_files):
    feature_dict = torch.load(f)
    if feature_dict['i3d_feature_map'].shape[1] != 23:
        issues.append(f)
        continue
    if feature_dict['2d_cnn_feature_map'].shape[1] != 23:
        issues.append(f)

100%|██████████| 1787/1787 [06:51<00:00,  4.34it/s]
